<a href="https://www.kaggle.com/code/sanhithreddy/emoji-prediction-lstm-in-tensorflow?scriptVersionId=263510130" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [59]:
import numpy as np
import pandas as pd
import emoji

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Embedding

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [60]:
data = pd.read_csv('/kaggle/input/emojis-dataset/emoji_dataset.csv')
data.head(10)

,text,emoji
0,I am feeling so happy today,😊
1,Life is beautiful,😊
2,This made my day,😊
3,I'm smiling all the way,😊
4,Feeling blessed and positive,😊
5,Such a peaceful morning,😊
6,I love this sunshine,😊
7,Today is the best day ever,😊
8,I can't stop smiling,😊
9,Good vibes only,😊


In [61]:
data.shape

(279, 2)

In [62]:
data['emoji'].value_counts()

emoji
😊     35
😂     35
😢     34
😡     25
❤️    25
😎     25
🎉     25
🙏     25
🔥     25
😴     25
Name: count, dtype: int64

In [63]:
emoji_to_number = {
    "😊": 0,
    "😢": 1,
    "😡": 2,
    "😂": 3,
    "❤️": 4,
    "😎": 5,
    "🎉": 6,
    "🙏": 7,
    "🔥": 8,
    "😴": 9
}

# Convert emoji column to numbers
data["emoji_label"] = data["emoji"].map(emoji_to_number)

In [64]:
data

,text,emoji,emoji_label
0,I am feeling so happy today,😊,0
1,Life is beautiful,😊,0
2,This made my day,😊,0
3,I'm smiling all the way,😊,0
4,Feeling blessed and positive,😊,0
...,...,...,...
274,This had me in stitches,😂,3
275,Comedy gold,😂,3
276,I'm dying of laughter,😂,3
277,This meme is hilarious,😂,3


In [65]:
X = data['text'].values
from tensorflow.keras.utils import to_categorical

Y=data['emoji_label'].values

In [66]:
X = [s.replace("i'm", "i am") for s in X]


In [67]:
X


['I am feeling so happy today',
 'Life is beautiful',
 'This made my day',
 "I'm smiling all the way",
 'Feeling blessed and positive',
 'Such a peaceful morning',
 'I love this sunshine',
 'Today is the best day ever',
 "I can't stop smiling",
 'Good vibes only',
 'Finally achieved my goal',
 'This song makes me so cheerful',
 'Spreading positivity',
 'My heart feels light',
 'This is pure happiness',
 "I'm in a great mood",
 'Everything is awesome',
 'Life is good',
 'Smiling through it all',
 'I feel so energetic',
 'Having fun with friends',
 'Good day at work',
 'I feel optimistic',
 'Joy all around',
 'Love and light',
 'I miss you a lot',
 "I'm so sad right now",
 'Feeling heartbroken',
 'Why did this happen',
 "I'm crying inside",
 'My heart hurts',
 'This is so depressing',
 'Feeling lonely',
 'I just lost something important',
 'Wish you were here',
 "I'm in pain",
 'Sad songs on repeat',
 "I can't stop crying",
 "My tears won't stop",
 'Feeling down today',
 'This is unbeara

In [68]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

# Embeddings

In [69]:
file = open('/kaggle/input/glove-embedding/glove.6B.100d.txt', 'r', encoding = 'utf8')
content = file.readlines()
file.close()

# content

In [70]:
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype = float)

In [71]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index

In [72]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    return maxlen

maxlen = get_maxlen(Xtokens)
print(maxlen)

6


In [73]:
Xtokens = tokenizer.texts_to_sequences(X)
Xtrain = pad_sequences(Xtokens, maxlen = maxlen,  padding = 'post', truncating = 'post')

In [74]:
Ytrain = to_categorical(Y)

# Model

In [75]:
embed_size = 100
embedding_matrix = np.zeros((len(word2index) + 1, embed_size))

for word, i in word2index.items():
    # Use .get() to avoid KeyError
    embed_vector = embeddings.get(word)
    if embed_vector is not None:
        embedding_matrix[i] = embed_vector
    # else: the row remains zeros

In [76]:
embedding_matrix

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [-0.57058  ,  0.44183  ,  0.70102  , ..., -0.66102  ,  0.47197  ,
         0.37253  ],
       ...,
       [ 0.70406  ,  0.21847  ,  0.50543  , ..., -0.0043049, -0.71895  ,
         0.4682   ],
       [ 0.18779  ,  0.60083  ,  0.36195  , ..., -0.41267  , -0.30172  ,
         0.48881  ],
       [-1.077    , -0.42305  ,  0.72816  , ...,  0.25724  , -0.025671 ,
         0.0526   ]])

In [77]:
num_classes = Ytrain.shape[1]  # 10 in your case

model = Sequential([
    Embedding(
        input_dim=len(word2index) + 1,
        output_dim=embed_size,
        weights=[embedding_matrix],
        trainable=False
    ),
    LSTM(units=16, return_sequences=True),
    LSTM(units=4),
    Dense(num_classes, activation='softmax')  # <- match number of classes
])


In [78]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [83]:
model.fit(Xtrain, Ytrain, epochs = 150)

Epoch 1/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9881 - loss: 0.0437
Epoch 2/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9952 - loss: 0.0347
Epoch 3/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9918 - loss: 0.0434
Epoch 4/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9982 - loss: 0.0290
Epoch 5/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9912 - loss: 0.0479
Epoch 6/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9928 - loss: 0.0351
Epoch 7/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9892 - loss: 0.0371
Epoch 8/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9873 - loss: 0.0425
Epoch 9/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9816 - loss: 0.0568
Epoch 10/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9924 - loss: 0.0395
Epoch 11/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9964 - loss: 0.0311
Epoch 12/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9968 - lo

In [84]:
def label_to_emoji(label):
    mapping = {
        0: "😀", 1: "😢", 2: "😡", 3: "😱", 4: "😍",
        5: "😐", 6: "🤔", 7: "😴", 8: "🤢", 9: "😎"
    }
    return mapping.get(label, "")


In [87]:
test = ["I feel good"]

test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis = 1)

for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
I feel good 😀
